In [2]:
import carla 
import math 
import random 
import time 
import numpy as np
import cv2

# Connect the client and set up bp library and spawn points
client = carla.Client('localhost', 2000) 
world = client.get_world()
bp_lib = world.get_blueprint_library() 
spawn_points = world.get_map().get_spawn_points() 

In [3]:
# istanzia macchina
vehicle_bp = bp_lib.find('vehicle.lincoln.mkz_2020') 
vehicle = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))

# sposta il pov dello spettatore alle spalle dell'auto
spectator = world.get_spectator() 
transform = carla.Transform(vehicle.get_transform().transform(carla.Location(x=-4,z=2.5)),vehicle.get_transform().rotation) 
spectator.set_transform(transform)

In [4]:
#recupera la telecamera rgb dalla blueprint library
camera_bp = bp_lib.find('sensor.camera.rgb') 
camera_init_trans = carla.Transform(carla.Location(z=2)) 
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle)

time.sleep(0.2)
spectator.set_transform(camera.get_transform()) #sposta il pov dello spettatore a quello della telecamera
camera.destroy()

True

In [9]:
# istanzia camera rgb legandola al veicolo, la camera si sposterà insieme al veicolo conservando le coordinate di spawn rispetto al suolo
camera_init_trans = carla.Transform(carla.Location(x=2.5, z=0.7))
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle)

# callback per conversione e formattazione dei dati raw della telecamera in modo che siano fruibili esternamente                         
def camera_callback(image, data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

# recupera le dimensioni dell'immagine dal sensore                       
image_w = camera_bp.get_attribute("image_size_x").as_int()
image_h = camera_bp.get_attribute("image_size_y").as_int()
#inizializza una mappa vuota per memorizzare l'immagine mediante callback
camera_data = {'image': np.zeros((image_h, image_w, 4))}

# registrazione della telecamera
camera.listen(lambda image: camera_callback(image, camera_data))

vehicle.set_autopilot(True)


In [10]:
# finestra open cv per mvisualizazione real time del movimento della macchina
cv2.namedWindow('RGB Camera', cv2.WINDOW_AUTOSIZE)
cv2.imshow('RGB Camera', camera_data['image'])
cv2.waitKey(1)

# loop infinito per finchè l'utente non preme 'q', per mostrare la finestra opencv
while True:
    
    # Imshow renders sensor data to display
    cv2.imshow('RGB Camera', camera_data['image'])
    
    # Quit if user presses 'q'
    if cv2.waitKey(1) == ord('q'):
        break

# Close OpenCV window when finished
cv2.destroyAllWindows()
cv2.stop()

AttributeError: module 'cv2' has no attribute 'stop'